In [16]:
from methods import load_data, select_route_data, group_nodes, group_links, obtain_routes, select_volume_data, routes_volume_join, drop_duplicates, export_to_json, qgis_json_format
import pathlib

In [17]:
LINK_INPUT = "anyoldinput"
LINK_OUTPUT = "anyoldoutput"

In [18]:
file = pathlib.Path.cwd() / "Enfield Saturn Routes AM test.xlsx"

In [19]:
#Load raw data
strategic_raw_data, qgis_table = load_data(file, file)

In [20]:
nodes = select_route_data(strategic_raw_data)
ogv_nodes = select_route_data(strategic_raw_data, ogv=True) #Extract routes for user class 9 (OGVs)

In [21]:
nodes_grouped=group_nodes(nodes)
ogv_nodes_grouped=group_nodes(ogv_nodes)

In [22]:
links = group_links(nodes_grouped)
ogv_links = group_links(ogv_nodes_grouped)

In [23]:
routes = obtain_routes(links, qgis_table)
ogv_routes=obtain_routes(ogv_links, qgis_table)

Separate gis sheet from same file

In [24]:
all_routes = routes + ogv_routes

In [25]:
volumes=select_volume_data(strategic_raw_data)

In [26]:
routes = routes_volume_join(routes, volumes)
ogv_routes = routes_volume_join(ogv_routes, volumes)

In [27]:
unique_routes = drop_duplicates(routes)
ogv_unique_routes = drop_duplicates(ogv_routes)

In [28]:
qgis_routes = qgis_json_format(unique_routes)
ogv_qgis_routes = qgis_json_format(ogv_unique_routes)

In [29]:
export_to_json("routes", qgis_routes)
export_to_json("ogv_routes", ogv_qgis_routes)